In [1]:
# v4: Parametizes all str inputs, restructures the code for readability and Pythonic syntax

In [1]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime as dt
from datetime import timedelta as td

In [2]:
arg_date = '2022-12-28'
src_format = '%Y-%m-%d'
src_bucket = 'xetra-1234'
trg_bucket = 'etl-project-data'
columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
key = 'xetra_daily_report_' + dt.today().strftime("%Y%m%d_%H:%M:%S") + '.parquet'

In [3]:
arg_date_dt = dt.strptime(arg_date, src_format).date() - td(days=1)

In [4]:
s3 = boto3.resource('s3')
bucket = s3.Bucket(src_bucket)
objects = [obj for obj in bucket.objects.all() if dt.strptime(obj.key.split('/')[0], src_format).date() >= arg_date_dt]

In [5]:
def csv_to_df(filename):
    csv_obj = bucket.Object(key=filename).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=',')
    return df

df_all = pd.concat([csv_to_df(obj.key) for obj in objects], ignore_index=True)

In [6]:
df_all = df_all.loc[:, columns]

In [7]:
df_all.dropna(inplace=True)

## Transformations

### Get opening price per ISIN and day

In [8]:
df_all['OpeningPrice'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')

### Get closing price per ISIN and day

In [9]:
df_all['ClosingPrice'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['EndPrice'].transform('last')

In [10]:
df_all

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,OpeningPrice,ClosingPrice
0,AT0000A0E9W5,2022-12-27,08:00,14.02,14.27,14.02,14.27,1466,14.02,14.01
1,DE000A0D6554,2022-12-27,08:00,13.84,13.85,13.79,13.82,26486,13.84,14.01
2,DE000A0D9PT0,2022-12-27,08:00,197.35,197.55,197.0,197.25,2150,197.35,209.7
3,DE000A0JL9W6,2022-12-27,08:00,63.0,63.25,62.9,62.9,2077,63.0,66.25
4,DE000A0LD6E6,2022-12-27,08:00,65.6,65.6,65.6,65.6,1801,65.6,65.95
...,...,...,...,...,...,...,...,...,...,...
664085,GB00BLD4ZP54,2022-12-31,16:46,19.324,19.324,19.324,19.324,0,18.508,19.324
664086,LU1923627332,2022-12-31,16:52,12.4,12.4,12.4,12.4,2645,10.8,12.4
664087,US98956P1021,2022-12-31,20:30,113.1,113.1,113.1,113.1,0,112.5,113.1
664088,US9224171002,2022-12-31,20:30,24.6,24.6,24.6,24.6,0,24.6,24.6


### Aggregations

In [ ]:
df_all = df_all.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('OpeningPrice', 'min'), 
                                                              closing_price_eur=('ClosingPrice', 'min'), 
                                                              minimum_price_eur=('MinPrice', 'min'),
                                                              maximum_price_eru=('MaxPrice','max'),
                                                              daily_traded_volume=('TradedVolume','sum'))

### Percent change from previous closing price

In [12]:
df_all['previous_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)

In [13]:
df_all['%_change_closing_price'] = (df_all['closing_price_eur'] - df_all['previous_closing_price']) / df_all['previous_closing_price'] * 100

In [14]:
df_all.drop(columns=['previous_closing_price'], inplace=True)

In [15]:
df_all = df_all.round(decimals=2)

In [16]:
df_all = df_all[df_all.Date >= arg_date]

## Write to s3

In [17]:
out_buffer = BytesIO()

In [18]:
df_all.to_parquet(out_buffer, index=False)

In [19]:
bucket_target = s3.Bucket(trg_bucket)
bucket_target.put_object(Body=out_buffer.getvalue(), Key=key)

s3.Object(bucket_name='etl-project-data', key='xetra_daily_report_20230213_21:40:47.parquet')

## Review contents of uploaded file to verify proper publishing

In [21]:
for obj in bucket_target.objects.all():
    print(obj.key)

xetra_daily_report_20230213_21:12:18.parquet
xetra_daily_report_20230213_21:40:47.parquet


In [22]:
prq_obj = bucket_target.Object(key='xetra_daily_report_20230213_21:40:47.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [23]:
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eru,daily_traded_volume,%_change_closing_price
0,AT000000STR1,2022-12-28,36.60,36.70,35.75,36.70,1773,-2.65
1,AT000000STR1,2022-12-29,36.60,36.70,35.75,36.70,1773,0.00
2,AT000000STR1,2022-12-30,36.60,36.70,35.75,36.70,1773,0.00
3,AT000000STR1,2022-12-31,36.60,36.70,35.75,36.70,1773,0.00
4,AT00000FACC2,2022-12-28,8.05,8.57,7.87,8.57,10205,2.51
...,...,...,...,...,...,...,...,...
12923,XS2376095068,2022-12-31,34.29,36.50,34.06,36.50,1000,0.00
12924,XS2434891219,2022-12-28,3.44,3.66,3.42,3.66,0,4.53
12925,XS2434891219,2022-12-29,3.44,3.66,3.42,3.66,0,0.00
12926,XS2434891219,2022-12-30,3.44,3.66,3.42,3.66,0,0.00
